# Eigenvalue problems

We solve the generalized eigenvalue problem

$$
A u = \lambda M u,
$$

where $A$ comes from $\int \nabla u \nabla v$, and $M$ from $\int u v$, on the space $H_0^1$

In [2]:
import netgen.gui
%gui tk
from netgen.geom2d import unit_square
from ngsolve import *
from math import pi
import scipy.linalg
from scipy import random

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

We setup a stiffness matrix $A$ and a mass matrix $M$, and declare a preconditioner for $A$:

In [3]:
fes = H1(mesh, order=4, dirichlet=[1,2,3,4])
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v))
pre = Preconditioner(a, "multigrid")

m = BilinearForm(fes)
m += SymbolicBFI(u*v)

a.Assemble()
m.Assemble()

u = GridFunction(fes)

The inverse iteration is

$$
u_{n+1} = A^{-1} M u_n,
$$

where the Rayleigh quotient
$$
\rho_n = \frac{\left \langle A u_n, u_n\right \rangle}{\left 
\langle M u_n, u_n\right \rangle}
$$

converges to the smallest eigenvalue, with rate of convergence $\lambda_1 / \lambda_2$.

The preconditioned inverse iteration (PINVIT), see [Knyazef+Neymeyr], replaces $A^{-1}$ by an approximate inverse $C^{-1}$:

$$
\rho_n = \frac{\left \langle A u_n, u_n\right \rangle}{\left \langle M u_n, u_n\right \rangle} \\
w_n = C^{-1} (A u_n - \rho M u_n) \\
u_{n+1} = u_n + \alpha w_n
$$

The optimal step-size $\alpha$ is found by minimizing the Rayleigh-quotient on a two-dimensional space:

$$
u_{n+1} = \operatorname{arg} \min_{v \in \{ u_n, w_n\}} \frac{\left \langle A v, v\right \rangle}{\left \langle M v, v\right \rangle} 
$$

This minimization problem can be solved by a small eigenvalue problem

$$
a y = \lambda m y
$$
with matrices
$$
a = 
\left( \begin{array}{cc}
\left \langle  A u_n, u_n \right \rangle &
\left \langle  A u_n, w_n \right \rangle \\
\left \langle  A w_n, u_n \right \rangle &
\left \langle  A w_n, w_n \right \rangle 
\end{array} \right) \quad
m = 
\left( \begin{array}{cc}
\left \langle  M u_n, u_n \right \rangle &
\left \langle  M u_n, w_n \right \rangle \\
\left \langle  M w_n, u_n \right \rangle &
\left \langle  M w_n, w_n \right \rangle 
\end{array} \right)
$$

Then, the new iterate is

$$
u_{n+1} = y_1 u_n + y_2 w_n
$$

where $y$ is the eigenvector corresponding to the smaller eigenvalue.

### Implementation in NGSolve

Create some help vectors:

In [4]:
r = u.vec.CreateVector()
w = u.vec.CreateVector()
Mu = u.vec.CreateVector()
Au = u.vec.CreateVector()
Mw = u.vec.CreateVector()
Aw = u.vec.CreateVector()

random initial condition, zero on the boundary

In [5]:
r.FV().NumPy()[:] = random.rand(fes.ndof)
u.vec.data = Projector(fes.FreeDofs(), True) * r

# from random import random
# freedofs = fes.FreeDofs()
# for i in range(len(u.vec)):
#    u.vec[i] = random() if freedofs[i] else 0  

In [6]:
for i in range(20):
    Au.data = a.mat * u.vec
    Mu.data = m.mat * u.vec
    auu = InnerProduct(Au, u.vec)
    muu = InnerProduct(Mu, u.vec)
    # Rayleigh quotiont
    lam = auu/muu
    print (lam / (pi**2))
    # residual
    r.data = Au - lam * Mu
    w.data = pre.mat * r.data
    w.data = 1/Norm(w) * w
    Aw.data = a.mat * w
    Mw.data = m.mat * w

    # setup and solve 2x2 small eigenvalue problem
    asmall = Matrix(2,2)
    asmall[0,0] = auu
    asmall[0,1] = asmall[1,0] = InnerProduct(Au, w)
    asmall[1,1] = InnerProduct(Aw, w)
    msmall = Matrix(2,2)
    msmall[0,0] = muu
    msmall[0,1] = msmall[1,0] = InnerProduct(Mu, w)
    msmall[1,1] = InnerProduct(Mw, w)
    # print ("asmall =", asmall, ", msmall = ", msmall)
    
    
    eval,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    # print (eval, evec)
    u.vec.data = float(evec[0,0]) * u.vec + float(evec[1,0]) * w
    
Draw (u)

24.15512547442816
2.1334261571334943
2.003318167034182
2.0002975751328154
2.0000501126150816
2.0000122421519864
2.0000031933182894
2.000000848014699
2.0000002273680964
2.000000061393768
2.00000001667718
2.000000004568741
2.000000001277216
2.0000000003798983
2.0000000001347122
2.0000000000675984
2.000000000049199
2.0000000000441496
2.0000000000427622
2.0000000000423817


## Simultaneous iteration for several eigenvalues

Declare GridFunction with multiple components to store several eigenvectors.

In [7]:
num = 5
u = GridFunction(fes, multidim=num)

Create list of help vectors, random initialization of u vectors:

In [8]:
r = u.vec.CreateVector()
Av = u.vec.CreateVector()
Mv = u.vec.CreateVector()

vecs = []
for i in range(2*num):
    vecs.append (u.vec.CreateVector())

for v in u.vecs:
    r.FV().NumPy()[:] = random.rand(fes.ndof)
    v.data = Projector(fes.FreeDofs(), True) * r

Compute num residuals, and solve small eigenvalue problem on $2 num$-dimensional space

In [14]:
asmall = Matrix(2*num, 2*num)
msmall = Matrix(2*num, 2*num)
lams = num * [1]

for i in range(20):
    
    for j in range(num):
        vecs[j].data = u.vecs[j]
        r.data = a.mat * vecs[j] - lams[j] * m.mat * vecs[j]
        vecs[num+j].data = pre.mat * r

    for j in range(2*num):
        Av.data = a.mat * vecs[j]
        Mv.data = m.mat * vecs[j]
        for k in range(2*num):
            asmall[j,k] = InnerProduct(Av, vecs[k])
            msmall[j,k] = InnerProduct(Mv, vecs[k])

    ev,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    lams[:] = ev[0:num]
    print (i, ":", [lam/pi**2 for lam in lams])
    
    for j in range(num):
        u.vecs[j][:] = 0.0
        for k in range(2*num):
            u.vecs[j].data += float(evec[k,j]) * vecs[k]
            
Draw (u)

0 : [2.0000000000422347, 5.0000000046724136, 5.0000000050875961, 8.0000000479993219, 10.000000173093328]
1 : [2.0000000000422355, 5.0000000046722182, 5.0000000050876032, 8.000000047998725, 10.000000173093275]
2 : [2.0000000000422347, 5.0000000046726347, 5.0000000050876201, 8.0000000479988511, 10.000000173093298]
3 : [2.000000000042236, 5.0000000046723097, 5.0000000050877631, 8.000000047998455, 10.000000173093724]
4 : [2.0000000000422355, 5.0000000046723514, 5.000000005087502, 8.000000047998352, 10.000000173093554]
5 : [2.0000000000422355, 5.0000000046723549, 5.0000000050877551, 8.0000000479986912, 10.000000173093461]
6 : [2.0000000000422355, 5.0000000046721773, 5.0000000050876805, 8.0000000479982702, 10.000000173093298]
7 : [2.0000000000422351, 5.0000000046724171, 5.0000000050875961, 8.0000000479984603, 10.000000173093245]
8 : [2.000000000042236, 5.0000000046723061, 5.0000000050877027, 8.0000000479984816, 10.000000173093355]
9 : [2.000000000042236, 5.0000000046723017, 5.000000005087549

The *multidim-component* select in the *Visualization* dialog box allows to switch between the components of the multidim-GridFunction.